# Floquet optimization

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

In [ ]:
import os
import numpy as np
import qutip as qt
import nlopt
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Image, display
if not isnotebook:
    def display(args):
        pass

In [ ]:
import qocttools
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.hamiltonians as hamiltonians
import qocttools.floquet as floquet
import qocttools.target as target
qocttools.about()

In [ ]:
if "NP_RANDOM_SEED" in os.environ:
    seed = int(os.environ["NP_RANDOM_SEED"])
else:
    seed = 1

In [ ]:
data = []

# Model definition

In [ ]:
Sx = qt.jmat(1, "x")
Sy = qt.jmat(1, "y")
Sz = qt.jmat(1, "z")
Bs = 0.3
Nz = 1.00
Nxy = 0.05
Bd = 0.1
omega = 1.00
gamma = 0.2
beta = 3.0
d = 3
dim = d**2

In [ ]:
def system_definition():
    H0 = -Bs * Sz + Nz * Sz**2 + Nxy * (Sx**2 - Sy**2)
    Vx = -Bd * Sx
    Vy = -Bd * Sy
    A = []
    e, psi = H0.eigenstates()
    for i in range(d):
        for j in range(d):
            if j == i:
                continue
            gammaij = gamma * np.exp(-beta*e[j]) / (np.exp(-beta*e[i])+np.exp(-beta*e[j]))
            A.append( np.sqrt(gammaij) * psi[j] * psi[i].dag())
    return H0, [Vx, Vy], A, e, psi

H0, V, A, e, psi = system_definition()

In [ ]:
print("Field-free eigenvalues = {}".format(e))

# Basic freequency and period

In [ ]:
#omega_eV = 2.0
#omega = omega_eV / eV
omega = 3.0
T = (2.0*np.pi/omega)

print("The periodic driving frequency is {} eV.".format(omega))
print("The Floquet period is {} a.u".format(T))


# Zero-field calculation

In [ ]:
a0 = 0.0

def Axref(t, args):
    return a0 * np.sin(omega * t)

def Ayref(t, args):
    return a0 * np.cos(omega * t)

In [ ]:
H = [H0, [V[0], Axref], [V[1], Ayref]]

In [ ]:
#?qt.FloquetBasis

In [ ]:
epsilon0 = floquet.epsilon(H, T)
print("Field-free Floquet pseudoenergies = {}".format(epsilon0))

# Non-zero field (reference) calculations

In [ ]:
a0 = 5.0

In [ ]:
#print(qt.floquet_modes(H, T)[1])
epsilon_ref = floquet.epsilon(H, T)
print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
print("Field-free Floquet pseudoenergies = {}".format(epsilon0))
print("Diff with the field-free eigenvalues = {}".format(epsilon_ref-epsilon0))

# Pulse definitions

In [ ]:
times = np.linspace(0, T, 100)
maxamp = a0 * np.sqrt(T) / 2

read_initial_guess_from_disk = True

In [ ]:
if not read_initial_guess_from_disk:
    
    M = 5
    random_initial_pulse = True

    if random_initial_pulse:
        random_bound = 1.0 * maxamp
        u1 = np.zeros(2*M+1)
        u1[1:] = np.random.uniform(low = -1.0, high = 1.0, size = 2*M)
        u1[1:] = random_bound * u1[1:]
        u2 = np.zeros(2*M+1)
        u2[1:] = np.random.uniform(low = -1.0, high = 1.0, size = 2*M)
        u2[1:] = random_bound * u2[1:]
    else:
        u1 = np.zeros(2*M+1)
        u1[7] = maxamp
        u2 = np.zeros(2*M+1)
        u2[8] = maxamp
    
    Ax0 = pulses.pulse("fourier", T, u = u1)
    Ay0 = pulses.pulse("fourier", T, u = u2)
    Axopt = pulses.pulse("fourier", T, u = u1)
    Ayopt = pulses.pulse("fourier", T, u = u2)
    #Axr = pulses.pulse("fourier", T, u = u1)
    #Ayr = pulses.pulse("fourier", T, u = u2)
    #uopt = Axr.fitparams(Axref(times, None), times, u1)
    #Axr.set_parameters(uopt)
    #uopt = Ayr.fitparams(Ayref(times, None), times, u2)
    #Ayr.set_parameters(uopt)
    Ax0.print('Ax0')
    Ay0.print('Ay0')
else:
    Ax0 = pulses.read_pulse('Ax0')
    Ay0 = pulses.read_pulse('Ay0')
    Axopt = pulses.read_pulse('Ax0')
    Ayopt = pulses.read_pulse('Ay0')

#Axopt.set_constraint('zero_average')
#Ayopt.set_constraint('zero_average')

In [ ]:
fig = plt.figure(figsize=(5,4), facecolor = 'red', frameon = True, edgecolor = 'black', linewidth = 5)
ax = fig.add_axes([0.15, 0.15, 0.8, 0.8])

times = np.linspace(0, T, 100)

ax.plot(times/T, Axref(times, None), label = r"$A^{\rm ref}_x(t)$")
ax.plot(times/T, Ayref(times, None), label = r"$A^{\rm ref}_y(t)$")

ax.plot(times/T, Ax0.fu(times), label = r"$A^{\rm 0}_x(t)$")
ax.plot(times/T, Ay0.fu(times), label = r"$A^{\rm 0}_y(t)$")

#ax.plot(times/T, Axref(times, None))
#ax.plot(times/T, Ayref(times, None))

ax.legend()
ax.set_xlim(left = 0, right = 1)

fname = 'reference-fields'
fig.savefig(fname + '.pdf')
fig.savefig(fname + '.png')
plt.close(fig)
display(Image(fname + '.png'))

In [ ]:
u0 = pulses.pulse_collection_get_parameters([Ax0, Ay0])

# Hamiltonian

In [ ]:
H = hamiltonians.hamiltonian(H0, V)

In [ ]:
def Hfunc(t, args):
    fx = args["f"][0]
    fy = args["f"][1]
    return H0 + fx(t)*V[0] + fy(t)*V[1]

def Vfuncx(t, args):
    return V[0]

def Vfuncy(t, args):
    return V[1]

H_ = hamiltonians.hamiltonian(Hfunc, [Vfuncx, Vfuncy])

# Initial-guess calculations

In [ ]:
u = u0.copy()

In [ ]:
epsilon_ig = floquet.epsilon3(H, [Ax0, Ay0], u, T)
print("Initial-guess Floquet pseudoenergies = {}".format(epsilon_ig))
print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
print("Diff = {}".format(epsilon_ig-epsilon_ref))

In [ ]:
epsilon_ig = floquet.epsilon3(H_, [Ax0, Ay0], u, T)
print("Initial-guess Floquet pseudoenergies = {}".format(epsilon_ig))
print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
print("Diff = {}".format(epsilon_ig-epsilon_ref))

# Target definition

In [ ]:
targeteps = epsilon_ref.reshape(1, 3)

In [ ]:
target_def = 0

if target_def == 0:
    
    def f(eps):
        cte = 1.0
        fval = 0.0
        nkpoints = eps.shape[0]
        targete = targeteps
        dim = eps.shape[1]
        fval = 0.0
        for k in range(nkpoints):
            for alpha in range(dim):
                fval = fval - cte * (eps[k, alpha] - targete[k, alpha])**2
        return fval
    
    def dfdepsilon(eps):
        cte = 1.0
        nkpoints = eps.shape[0]
        targete = targeteps
        dim = eps.shape[1]
        dfval = np.zeros((nkpoints, dim))
        for k in range(nkpoints):
            for alpha in range(dim):
                dfval[k, alpha] = - 2.0 * cte * (eps[k, alpha]-targete[k, alpha])
        return dfval
    
elif target_def == 1:

    def f(eps):
        fval = 0.0
        dim = eps.shape[1]
        nkpoints = eps.shape[0]
        for k in range(nkpoints):
            fval = fval + eps[k, 0]-eps[k, 2]
        return fval

    def dfdepsilon(eps):
        dim = eps.shape[1]
        nkpoints = eps.shape[0]
        dfval = np.zeros((nkpoints, dim))
        for k in range(nkpoints):
            dfval[k, 0] = 1.0
            dfval[k, 1] = 0.0
            dfval[k, 2] = -1.0
        return dfval

# Optimization: Perturbation-theory formula

In [ ]:
u = u0.copy()
pulses.pulse_collection_set_parameters([Axopt, Ayopt], u)

In [ ]:
U0 = qt.qeye(3)
U0set = []
#for k in range(targetset.nkpoints):
U0set.append(U0)

In [ ]:
tg = target.Target('floquet', targeteps = targeteps,
                   T = T, fepsilon = f, dfdepsilon = dfdepsilon)
#tg = target.Target('floquet', targeteps = targeteps, T = T)

In [ ]:
opt = qoct.Qoct(H, T, times.shape[0], tg, [Axopt, Ayopt], U0set,
                floquet_mode = 'pt')
print("G(u) = {} (initial guess)".format(opt.gfunc(u)))

In [ ]:
check_gradient = True
if check_gradient:
    #u = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    derqoct, dernum, error, elapsed_time = opt.check_grad(u)
    print("QOCT calculation: \t{}".format(derqoct))
    print("Ridders calculation: \t{} +- {}".format(dernum, error))
    data.append(derqoct)

optimize = False

if optimize:
    x, optval, res = opt.maximize(maxeval = 100,
                                  verbose = True,
                                  #tolerance = -1.0,
                                  #algorithm = nlopt.LD_MMA,
                                  algorithm = nlopt.LD_SLSQP,
                                  #algorithm = nlopt.LN_BOBYQA,
                                  #algorithm = nlopt.GD_STOGO,
                                  #algorithm = nlopt.LD_LBFGS,
                                  upper_bounds = 1 * np.abs(maxamp * np.ones_like(u)),
                                  lower_bounds = -1 * np.abs(maxamp * np.ones_like(u)))
    data.append(optval)
    uopt = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    print(opt.gfunc(uopt))
    epsilon_opt1 = floquet.epsilon3(H, [Axopt, Ayopt], uopt, T)
    print("Optimized Floquet pseudoenergies = {}".format(epsilon_opt1))
    print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
    print("Diff = {}".format(epsilon_opt1-epsilon_ref))

# Optimization: Perturbation-theory formula, Hamiltonian-as-a-function

In [ ]:
u = u0.copy()
pulses.pulse_collection_set_parameters([Axopt, Ayopt], u)

In [ ]:
U0 = qt.qeye(3)
U0set = []
#for k in range(targetset.nkpoints):
U0set.append(U0)

In [ ]:
tg = target.Target('floquet', targeteps = targeteps,
                   T = T, fepsilon = f, dfdepsilon = dfdepsilon)
#tg = target.Target('floquet', targeteps = targeteps, T = T)

In [ ]:
opt = qoct.Qoct(H_, T, times.shape[0], tg, [Axopt, Ayopt], U0set,
                solve_method = 'sesolve',
                floquet_mode = 'pt')
print("G(u) = {} (initial guess)".format(opt.gfunc(u)))

In [ ]:
check_gradient = True
if check_gradient:
    #u = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    derqoct, dernum, error, elapsed_time = opt.check_grad(u)
    print("QOCT calculation: \t{}".format(derqoct))
    print("Ridders calculation: \t{} +- {}".format(dernum, error))
    data.append(derqoct)

In [ ]:
optimize = False

if optimize:
    x, optval, res = opt.maximize(maxeval = 100,
                                  verbose = True,
                                  #tolerance = -1.0,
                                  #algorithm = nlopt.LD_MMA,
                                  algorithm = nlopt.LD_SLSQP,
                                  #algorithm = nlopt.LN_BOBYQA,
                                  #algorithm = nlopt.GD_STOGO,
                                  #algorithm = nlopt.LD_LBFGS,
                                  upper_bounds = 1 * np.abs(maxamp * np.ones_like(u)),
                                  lower_bounds = -1 * np.abs(maxamp * np.ones_like(u)))
    data.append(optval)
    uopt = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    print(opt.gfunc(uopt))
    epsilon_opt1 = floquet.epsilon3(H, [Axopt, Ayopt], uopt, T)
    print("Optimized Floquet pseudoenergies = {}".format(epsilon_opt1))
    print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
    print("Diff = {}".format(epsilon_opt1-epsilon_ref))

# Optimization: QOCT formula

In [ ]:
u = u0.copy()
pulses.pulse_collection_set_parameters([Axopt, Ayopt], u)

In [ ]:
U0 = qt.qeye(3)
U0set = []
#for k in range(targetset.nkpoints):
U0set.append(U0)

In [ ]:
#tg = target.Target('floquet', targeteps = targeteps, T = T)
tg = target.Target('floquet', targeteps = targeteps,
                   T = T, fepsilon = f, dfdepsilon = dfdepsilon)

In [ ]:
opt = qoct.Qoct(H, T, times.shape[0], tg, [Axopt, Ayopt], U0set,
                floquet_mode = 'qoct')
print("G(u) = {} (initial guess)".format(opt.gfunc(u)))

In [ ]:
check_gradient = True
if check_gradient:
    #u = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    derqoct, dernum, error, elapsed_time = opt.check_grad(u)
    print("QOCT calculation: \t{}".format(derqoct))
    print("Ridders calculation: \t{} +- {}".format(dernum, error))
    data.append(derqoct)

In [ ]:
optimize = False

if optimize:
    x, optval, res = opt.maximize(maxeval = 100,
                                  verbose = True,
                                  #tolerance = -1.0,
                                  #algorithm = nlopt.LD_MMA,
                                  algorithm = nlopt.LD_SLSQP,
                                  #algorithm = nlopt.LN_BOBYQA,
                                  #algorithm = nlopt.GD_STOGO,
                                  #algorithm = nlopt.LD_LBFGS,
                                  upper_bounds = 1 * np.abs(maxamp * np.ones_like(u)),
                                  lower_bounds = -1 * np.abs(maxamp * np.ones_like(u)))
    data.append(optval)
    uopt = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    print(opt.gfunc(uopt))
    epsilon_opt2 = floquet.epsilon3(H, [Axopt, Ayopt], uopt, T)
    print("Optimized Floquet pseudoenergies = {}".format(epsilon_opt2))
    print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
    print("Diff = {}".format(epsilon_opt2-epsilon_ref))

# Optimization: QOCT formula, Hamiltonian-as-a-function

In [ ]:
u = u0.copy()
pulses.pulse_collection_set_parameters([Axopt, Ayopt], u)

In [ ]:
U0 = qt.qeye(3)
U0set = []
#for k in range(targetset.nkpoints):
U0set.append(U0)

In [ ]:
#tg = target.Target('floquet', targeteps = targeteps, T = T)
tg = target.Target('floquet', targeteps = targeteps,
                   T = T, fepsilon = f, dfdepsilon = dfdepsilon)

In [ ]:
opt = qoct.Qoct(H_, T, times.shape[0], tg, [Axopt, Ayopt], U0set,
                solve_method = 'sesolve',
                floquet_mode = 'qoct')
print("G(u) = {} (initial guess)".format(opt.gfunc(u)))

In [ ]:
check_gradient = True
if check_gradient:
    #u = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    derqoct, dernum, error, elapsed_time = opt.check_grad(u)
    print("QOCT calculation: \t{}".format(derqoct))
    print("Ridders calculation: \t{} +- {}".format(dernum, error))
    data.append(derqoct)

In [ ]:
optimize = False

if optimize:
    x, optval, res = opt.maximize(maxeval = 100,
                                  verbose = True,
                                  #tolerance = -1.0,
                                  #algorithm = nlopt.LD_MMA,
                                  algorithm = nlopt.LD_SLSQP,
                                  #algorithm = nlopt.LN_BOBYQA,
                                  #algorithm = nlopt.GD_STOGO,
                                  #algorithm = nlopt.LD_LBFGS,
                                  upper_bounds = 1 * np.abs(maxamp * np.ones_like(u)),
                                  lower_bounds = -1 * np.abs(maxamp * np.ones_like(u)))
    data.append(optval)
    uopt = pulses.pulse_collection_get_parameters([Axopt, Ayopt])
    print(opt.gfunc(uopt))
    epsilon_opt2 = floquet.epsilon3(H, [Axopt, Ayopt], uopt, T)
    print("Optimized Floquet pseudoenergies = {}".format(epsilon_opt2))
    print("Reference Floquet pseudoenergies = {}".format(epsilon_ref))
    print("Diff = {}".format(epsilon_opt2-epsilon_ref))

# Non-equilibrium steady-states (NESS) optimization

In [ ]:
Ham = hamiltonians.hamiltonian(H0, V, A)
Ham__ = hamiltonians.toliouville(Ham)

In [ ]:
op1 = Sz
op2 = Sx*Sy+Sy*Sx
target_operator = op1

In [ ]:
omega0 = 0.5
T = (2.0*np.pi/omega0)
nts = 100
times = np.linspace(0, T, nts + 1)

In [ ]:
def pulse_definition(T, p, bound = 4.0, seed = 0):
    if seed >= 0:
        np.random.seed(seed)
        u = (bound-(-bound)) * np.random.random_sample(p) + (-bound)
        g1 = pulses.pulse("fourier", T, u)
        u = (bound-(-bound)) * np.random.random_sample(p) + (-bound)
        g2 = pulses.pulse("fourier", T, u)
    else:
        #M = p
        K = 1
        u = np.zeros(p)
        u[K] = bound #np.sqrt(T)/2
        g1 = pulses.pulse("fourier", T, u)
        u = np.zeros(p)
        u[K+1] = bound #np.sqrt(T)/2
        g2 = pulses.pulse("fourier", T, u)
    return [g1, g2]


def pulse_set_new(g, bound = 4.0, seed = 0):
    np.random.seed(seed)
    p = g[0].u.shape[0]
    u = (bound-(-bound)) * np.random.random_sample(p) + (-bound)
    g[0].set_parameters(u)
    u = (bound-(-bound)) * np.random.random_sample(p) + (-bound)
    g[1].set_parameters(u)

In [ ]:
M = 4
bound = 4.0
g = pulse_definition(T, 2*M+1, bound = bound, seed = -1)
gref = pulse_definition(T, 2*M+1, bound = bound, seed = -1)
u = pulses.pulse_collection_get_parameters(g)
pulses.pulse_collection_set_parameters(gref, u)

In [ ]:
tg = target.Target('floquet', operator = target_operator, T = T)
opt = qoct.Qoct(Ham, T, nts, tg, g, None, floquet_mode = 'ness')

In [ ]:
print("G(u) = {}".format(opt.gfunc(u)))
print("G(u=0) = {}".format(opt.gfunc(np.zeros_like(u))))

In [ ]:
check_gradient = True
if check_gradient:
    derqoct, dernum, error, elapsed_time = opt.check_grad(u)
    print("QOCT calculation: \t{}".format(derqoct))
    print("Ridders calculation: \t{} +- {}".format(dernum, error))
    data.append(derqoct)

# Output data

In [ ]:
with open("data", "w") as datafile:
    for i in data:
        datafile.write("{:.14e}\n".format(i))